In [24]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
df = pd.read_excel(r'../dataset/2023 Pollution Inventory Dataset V2.xlsx',skiprows=9)
df.head()

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR
0,UP3133UK,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Chequer Tree Farm And Coursehorn Farm Poultry ...,TN17 3PN,578630.0,134230.0,"Kent, South London and East Sussex",Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
1,QP3133UM,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Winfarthing Farm Poultry Unit Short Green Winf...,IP22 2EE,610040.0,286300.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
2,UP3133UK,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Chequer Tree Farm And Coursehorn Farm Poultry ...,TN17 3PN,578630.0,134230.0,"Kent, South London and East Sussex",Air,Particulate matter - PM10,1000.0,0,Agriculture,Intensive Farming
3,QP3133UM,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Winfarthing Farm Poultry Unit Short Green Winf...,IP22 2EE,610040.0,286300.0,East Anglia,Air,Particulate matter - PM10,1000.0,0,Agriculture,Intensive Farming
4,AP3833US,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Martham East Farm Poultry Unit Rollesby Road M...,NR29 4SL,645750.0,317400.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming


In [25]:
#dummy variables for certain columns
categorical_columns = ["EA AREA NAME", "ROUTE NAME", "REGULATED INDUSTRY SECTOR", "REGULATED INDUSTRY SUB SECTOR"]

for col in categorical_columns:
    unique_values = {value: idx for idx, value in enumerate(sorted(df[col].unique()))}
    df[col + " LABEL"] = df[col].map(unique_values)

#gases to be removed
remove_gases = [
    "Sulphur oxides (SO2 and SO3) as SO2", "Carbon monoxide", "Carbon dioxide", 
    "Carbon Dioxide From Qualifying Renewable Fuel Sources", "Methane", 
    "Nitrogen - as total N", "Nitrogen oxides (NO and NO2) as NO2", "Nitrous oxide", 
    "Sulphur hexafluoride", "Hydrogen cyanide", "Fluorine and inorganic fluorine compounds - as HF", 
    "Chlorofluorocarbons (CFCs)", "Hydrochlorofluorocarbons (HCFCs)", "Hydrofluorocarbons (HFCs)", 
    "Perfluorocarbons (PFCs)"]
#removing from dataset
df_no_gas = df[~df["SUBSTANCE NAME"].isin(remove_gases)]

In [26]:
#drop rows where quantity released is 0 or "Below Reporting Threshold"
df_no_gas_cleaned = df_no_gas[~df_no_gas["QUANTITY RELEASED (kg)"].isin([0, "Below Reporting Threshold"])]
df_no_gas_cleaned["ROUTE NAME"] = df_no_gas_cleaned["ROUTE NAME"].str.strip().str.lower()
df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"] = df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"].str.strip().str.lower()


#reset index after dropping rows
df_no_gas_cleaned.reset_index(drop=True, inplace=True)
df_no_gas_cleaned.head()

C:\Users\andre\AppData\Local\Temp\ipykernel_11148\4124804988.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_gas_cleaned["ROUTE NAME"] = df_no_gas_cleaned["ROUTE NAME"].str.strip().str.lower()
C:\Users\andre\AppData\Local\Temp\ipykernel_11148\4124804988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"] = df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"].str.strip().str.lower()


,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR,EA AREA NAME LABEL,ROUTE NAME LABEL,REGULATED INDUSTRY SECTOR LABEL,REGULATED INDUSTRY SUB SECTOR LABEL
0,BT5873IU,"INTENSIVE FARMING; > 40,000 POULTRY",LJ FAIRBURN AND SON LTD,Ivy House Farm Poultry Unit Ivy House Farm Far...,LN13 9PL,546500.0,374790.0,Lincolnshire and Northamptonshire,air,Particulate matter - PM10,1000.0,374,agriculture,Intensive Farming,7,0,0,11
1,XP3539RM,"INTENSIVE FARMING; > 40,000 POULTRY",Ridgeway Foods Limited,Seisdon Poultry Unit Fox Road Shipley West Mid...,WV6 7EL,382570.0,295320.0,West Midlands,air,Particulate matter - PM10,1000.0,395,agriculture,Intensive Farming,12,0,0,11
2,TP3636NV,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Bowlands Farm Poultry Unit Tenterden Road Bidd...,TN27 8BH,585550.0,137170.0,"Kent, South London and East Sussex",air,Particulate matter - PM10,1000.0,618,agriculture,Intensive Farming,6,0,0,11
3,PP3030QE,"INTENSIVE FARMING; > 40,000 POULTRY",Hill Crest Eggs Limited,Cumberworth Road Cumberworth Lincolnshire,LN13 9LB,550370.0,374090.0,Lincolnshire and Northamptonshire,air,Particulate matter - PM10,1000.0,689,agriculture,Intensive Farming,7,0,0,11
4,HP3634CZ,"INTENSIVE FARMING; > 40,000 POULTRY",Watkins; Watkins,Stonehouse Farm Kempley Dymock Gloucestershire,GL18 2AT,367380.0,231840.0,West Midlands,air,Particulate matter - PM10,1000.0,800,agriculture,Intensive Farming,12,0,0,11


In [27]:
print(len(df))
print(df.columns)
#remove air substances
df_no_gas_cleaned = df_no_gas_cleaned[df_no_gas_cleaned["ROUTE NAME"] != "air"]

30110
Index(['AUTHORISATION ID / PERMIT ID', 'ACTIVITY DESCRIPTION', 'OPERATOR NAME',
       'SITE ADDRESS', 'SITE POSTCODE', 'EASTING', 'NORTHING', 'EA AREA NAME',
       'ROUTE NAME', 'SUBSTANCE NAME', 'REPORTING THRESHOLD (kg)',
       'QUANTITY RELEASED (kg)', 'REGULATED INDUSTRY SECTOR',
       'REGULATED INDUSTRY SUB SECTOR', 'EA AREA NAME LABEL',
       'ROUTE NAME LABEL', 'REGULATED INDUSTRY SECTOR LABEL',
       'REGULATED INDUSTRY SUB SECTOR LABEL'],
      dtype='object')


In [28]:
#show what each label represents
for col in categorical_columns:
    unique_values = {value: idx for idx, value in enumerate(sorted(df_no_gas_cleaned[col].unique()))}
    print(f"\nLabel mappings for {col}:")
    for value, label in unique_values.items():
        print(f"{label}: {value}")


Label mappings for EA AREA NAME:
0: Cumbria and Lancashire
1: Devon, Cornwall and the Isles of Scilly
2: East Anglia
3: East Midlands
4: Greater Manchester Merseyside and Cheshire
5: Hertfordshire and North London
6: Kent, South London and East Sussex
7: Lincolnshire and Northamptonshire
8: North East
9: Solent and South Downs
10: Thames
11: Wessex
12: West Midlands
13: Yorkshire

Label mappings for ROUTE NAME:
0: controlled waters
1: wastewater

Label mappings for REGULATED INDUSTRY SECTOR:
0: biowaste treatment
1: cement and minerals
2: chemicals
3: combustion
4: efw
5: food & drink
6: landfill
7: metals
8: nuclear
9: paper and textiles
10: refineries & fuel
11: waste treatment
12: water industry

Label mappings for REGULATED INDUSTRY SUB SECTOR:
0: Biowaste Treatment
1: Cement
2: Chemicals
3: EfW
4: Ferrous
5: Food & Drink
6: Hazardous Waste
7: Inert Landfill
8: Metals Recycling
9: Non Hazardous Landfill
10: Non-Ferrous
11: Non-Hazardous & Inert
12: Nuclear
13: Paper & Pulp
14: Pow

In [29]:
#convert Easting/Northing to Latitude/Longitude
in_proj = Proj(init='epsg:27700')  #British National Grid
out_proj = Proj(init='epsg:4326')  #WGS84 (Latitude/Longitude)

#CONVERSION
def convert_coordinates(row):
    lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])
    return pd.Series([lat, lon])

df_no_gas_cleaned[['Latitude', 'Longitude']] = df_no_gas_cleaned.apply(convert_coordinates, axis=1)

C:\Users\andre\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\andre\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\andre\AppData\Local\Temp\ipykernel_11148\3859296045.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = tr

In [30]:
#create new excel files
df_no_gas.to_excel("pollution_no_gas.xlsx",index=False)
df_no_gas_cleaned.to_excel("pollution_no_gas_cleaned.xlsx",index=False)

In [31]:
print(df_no_gas_cleaned[df_no_gas_cleaned["ROUTE NAME"] == "Land"])


Empty DataFrame
Columns: [AUTHORISATION ID / PERMIT ID, ACTIVITY DESCRIPTION, OPERATOR NAME, SITE ADDRESS, SITE POSTCODE, EASTING, NORTHING, EA AREA NAME, ROUTE NAME, SUBSTANCE NAME, REPORTING THRESHOLD (kg), QUANTITY RELEASED (kg), REGULATED INDUSTRY SECTOR, REGULATED INDUSTRY SUB SECTOR, EA AREA NAME LABEL, ROUTE NAME LABEL, REGULATED INDUSTRY SECTOR LABEL, REGULATED INDUSTRY SUB SECTOR LABEL, Latitude, Longitude]
Index: []


In [32]:
len(df_no_gas_cleaned)

2995